<a href="https://colab.research.google.com/github/farshidehkordi/Homework2_AI/blob/main/TP2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os


# List files in the current directory
print(os.listdir())


['.config', 'drive', 'sample_data']


In [5]:
import os

# List files in the current directory
print(os.listdir())

# Check if the uploaded file exists
if 'CUB_200_2011.tar' in os.listdir():
    print("File exists!")
else:
    print("File not found.")


['.config', 'drive', 'sample_data']
File not found.


In [6]:
import tarfile

# open file
file = tarfile.open('/content/drive/MyDrive/CUB_200_2011.tar')



# extract files
file.extractall('./CUB_dataset')

# close file
file.close()


In [7]:
import os
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from shutil import copyfile


# Function to create a directory if it doesn't exist
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

# Function to separate images into training and test sets
def separate_train_test(dataset_path, train_path, test_path):
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        if classname.startswith('.'):
            continue
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.startswith('.'):
                continue
            file_path = os.path.join(dataset_path, classname, file)
            if i < 15:  # Number of images for the test set
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1
        class_index += 1







In [8]:
dataset_path = './CUB_dataset/CUB_200_2011/images'
train_path = './CUB_dataset/train'
test_path  = './CUB_dataset/test'

# Call the function to separate the dataset into training and test sets
separate_train_test(dataset_path, train_path, test_path)

ResNet18 using random initialization use the values from the training dataset to normalize the data.



In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Calculate mean and standard deviation from the training dataset
mean = torch.zeros(3)  # Initialize mean tensor
std = torch.zeros(3)   # Initialize standard deviation tensor

for image, _ in train_dataset:
    # Add pixel values of each channel to the mean tensor
    mean += torch.mean(image, dim=(1, 2))
    # Add standard deviation of each channel to the std tensor
    std += torch.std(image, dim=(1, 2))

# Calculate mean and standard deviation for the entire dataset
mean /= len(train_dataset)
std /= len(train_dataset)

print("Mean values for each channel:", mean)
print("Standard deviation values for each channel:", std)

# Define data transformations with normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
    transforms.Normalize(mean, std)                # Normalize images using computed mean and std
])

# Update the datasets with the new transformations
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with random initialization
model =resnet18(weights=None)

# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Mean values for each channel: tensor([0.4875, 0.5013, 0.4342])
Standard deviation values for each channel: tensor([0.1749, 0.1738, 0.1861])
Epoch [1/10], Loss: 5.301259991038929
Epoch [2/10], Loss: 5.129460662495006
Epoch [3/10], Loss: 4.960758075714112
Epoch [4/10], Loss: 4.817396018288353
Epoch [5/10], Loss: 4.677038718136874
Epoch [6/10], Loss: 4.517424056313255
Epoch [7/10], Loss: 4.3715684778040105
Epoch [8/10], Loss: 4.262436491359364
Epoch [9/10], Loss: 4.130862787420099
Epoch [10/10], Loss: 4.034660411314531
Accuracy on test set: 8.30%


ResNet18 using random initialization use the values from ImageNet

In [9]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
import torchvision.transforms as transforms

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with random initialization
model =resnet18(weights=None)

# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Epoch [1/10], Loss: 5.2730580954118205
Epoch [2/10], Loss: 5.052634617198597
Epoch [3/10], Loss: 4.857225154529918
Epoch [4/10], Loss: 4.69368781003085
Epoch [5/10], Loss: 4.560795438939874
Epoch [6/10], Loss: 4.443257312774659
Epoch [7/10], Loss: 4.346550821824508
Epoch [8/10], Loss: 4.221051257740368
Epoch [9/10], Loss: 4.122021278901534
Epoch [10/10], Loss: 4.016414707357233
Accuracy on test set: 9.10%


ResNet18 using pretrained model on CUB_200 dataset use the values from the training dataset to normalize the data, but freezing all the convolution parameters.



In [10]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18 , ResNet18_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Calculate mean and standard deviation from the training dataset
mean = torch.zeros(3)  # Initialize mean tensor
std = torch.zeros(3)   # Initialize standard deviation tensor

for image, _ in train_dataset:
    # Add pixel values of each channel to the mean tensor
    mean += torch.mean(image, dim=(1, 2))
    # Add standard deviation of each channel to the std tensor
    std += torch.std(image, dim=(1, 2))

# Calculate mean and standard deviation for the entire dataset
mean /= len(train_dataset)
std /= len(train_dataset)

print("Mean values for each channel:", mean)
print("Standard deviation values for each channel:", std)

# Define data transformations with normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
    transforms.Normalize(mean, std)                # Normalize images using computed mean and std
])

# Update the datasets with the new transformations
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the ResNet18 model with pre-trained weights
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Freeze conv1 and bn1 parameters
for param in model.conv1.parameters():
    param.requires_grad = False
for param in model.bn1.parameters():
    param.requires_grad = False

# Freeze convolution and batch norm parameters within each BasicBlock
for layer in [model.layer1, model.layer2, model.layer3, model.layer4]:
    for block in layer:
        for param in block.conv1.parameters():
            param.requires_grad = False
        for param in block.bn1.parameters():
            param.requires_grad = False
        for param in block.conv2.parameters():
            param.requires_grad = False
        for param in block.bn2.parameters():
            param.requires_grad = False


# Replace the fully connected layer with a new one (unfrozen)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')



Mean values for each channel: tensor([0.4875, 0.5013, 0.4342])
Standard deviation values for each channel: tensor([0.1749, 0.1738, 0.1861])
Epoch [1/10], Loss: 5.124327191439542
Epoch [2/10], Loss: 4.386361411701549
Epoch [3/10], Loss: 3.741461362838745
Epoch [4/10], Loss: 3.20933765151284
Epoch [5/10], Loss: 2.791107132651589
Epoch [6/10], Loss: 2.4660621179233897
Epoch [7/10], Loss: 2.2118477994745427
Epoch [8/10], Loss: 2.0008856977115976
Epoch [9/10], Loss: 1.8327548603578048
Epoch [10/10], Loss: 1.687067993770946
Accuracy on test set: 60.13%


Resnet18 using pretraind model for Freeze all layers use the values from ImageNet.

In [12]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet18, ResNet18_Weights
import matplotlib.pyplot as plt

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Load the pre-trained ResNet model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Freeze conv1 and bn1 parameters
for param in model.conv1.parameters():
    param.requires_grad = False
for param in model.bn1.parameters():
    param.requires_grad = False

# Freeze convolution and batch norm parameters within each BasicBlock
for layer in [model.layer1, model.layer2, model.layer3, model.layer4]:
    for block in layer:
        for param in block.conv1.parameters():
            param.requires_grad = False
        for param in block.bn1.parameters():
            param.requires_grad = False
        for param in block.conv2.parameters():
            param.requires_grad = False
        for param in block.bn2.parameters():
            param.requires_grad = False
# Adapt the fully connected layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(train_dataset.classes))  # Change the output size of the fully connected layer

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')




Epoch [1/10], Loss: 5.112201843261719
Epoch [2/10], Loss: 4.36550894390453
Epoch [3/10], Loss: 3.7233505916595457
Epoch [4/10], Loss: 3.1957089918309993
Epoch [5/10], Loss: 2.786312924298373
Epoch [6/10], Loss: 2.4620915985107423
Epoch [7/10], Loss: 2.2132579729773783
Epoch [8/10], Loss: 2.001765824664723
Epoch [9/10], Loss: 1.8350953769683838
Epoch [10/10], Loss: 1.692729663848877
Accuracy on test set: 59.40%


ResNet18 using the pre-trained model use the values from the training dataset to normalize the data, but only freezing the
parameters in "layer1" on CUB_200 dataset

In [11]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18 , ResNet18_Weights
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Calculate mean and standard deviation from the training dataset
mean = torch.zeros(3)  # Initialize mean tensor
std = torch.zeros(3)   # Initialize standard deviation tensor

for image, _ in train_dataset:
    # Add pixel values of each channel to the mean tensor
    mean += torch.mean(image, dim=(1, 2))
    # Add standard deviation of each channel to the std tensor
    std += torch.std(image, dim=(1, 2))

# Calculate mean and standard deviation for the entire dataset
mean /= len(train_dataset)
std /= len(train_dataset)

print("Mean values for each channel:", mean)
print("Standard deviation values for each channel:", std)

# Define data transformations with normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
    transforms.Normalize(mean, std)                # Normalize images using computed mean and std
])

# Update the datasets with the new transformations
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Freeze conv1 and bn1 parameters in layer1
for name, param in model.layer1.named_parameters():
    if 'conv1' in name or 'bn1' in name:
        param.requires_grad = False


# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Mean values for each channel: tensor([0.4875, 0.5013, 0.4342])
Standard deviation values for each channel: tensor([0.1749, 0.1738, 0.1861])
Epoch [1/10], Loss: 4.814942211671309
Epoch [2/10], Loss: 3.3240532094782047
Epoch [3/10], Loss: 2.4021008071032437
Epoch [4/10], Loss: 1.8132276994531804
Epoch [5/10], Loss: 1.3920872846516696
Epoch [6/10], Loss: 1.0875356143171138
Epoch [7/10], Loss: 0.8621930757435885
Epoch [8/10], Loss: 0.6700554305856878
Epoch [9/10], Loss: 0.5277180117910558
Epoch [10/10], Loss: 0.41452188372612
Accuracy on test set: 70.17%


ResNet18 using the pre-trained model use the values from the ImageNet to normalize the data, but only freezing the
parameters in "layer1" on CUB_200 dataset

In [9]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18 , ResNet18_Weights
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Freeze conv1 and bn1 parameters in layer1
for name, param in model.layer1.named_parameters():
    if 'conv1' in name or 'bn1' in name:
        param.requires_grad = False


# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 88.5MB/s]


Epoch [1/10], Loss: 4.765643106807362
Epoch [2/10], Loss: 3.3092130062796854
Epoch [3/10], Loss: 2.402303540923379
Epoch [4/10], Loss: 1.8049546397816052
Epoch [5/10], Loss: 1.3975991606712341
Epoch [6/10], Loss: 1.089412576718764
Epoch [7/10], Loss: 0.8604068558866328
Epoch [8/10], Loss: 0.6792901832407171
Epoch [9/10], Loss: 0.5245361735604026
Epoch [10/10], Loss: 0.4097622337666425
Accuracy on test set: 70.53%


ResNet18 using the pre-trained model on CUB_200 dataset use the values from the training dataset to normalize the data, but letting all the parameters (including the convolution
layers) be adjusted by backprop.

In [7]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize images
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


Epoch [1/10], Loss: 4.799591543024237
Epoch [2/10], Loss: 3.3336651143160734
Epoch [3/10], Loss: 2.419003915353255
Epoch [4/10], Loss: 1.8210269156369296
Epoch [5/10], Loss: 1.4034021405740218
Epoch [6/10], Loss: 1.0886978756297718
Epoch [7/10], Loss: 0.8522761667858471
Epoch [8/10], Loss: 0.6719024848937988
Epoch [9/10], Loss: 0.5179193783890117
Epoch [10/10], Loss: 0.4022182380611246
Accuracy on test set: 69.77%


ResNet18 using the pre-trained model on CUB_200 dataset use ImageNet to normalize the data, but letting all the parameters (including the convolution
layers) be adjusted by backprop

In [8]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
])

# Load the training and test datasets
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Calculate mean and standard deviation from the training dataset
mean = torch.zeros(3)  # Initialize mean tensor
std = torch.zeros(3)   # Initialize standard deviation tensor

for image, _ in train_dataset:
    # Add pixel values of each channel to the mean tensor
    mean += torch.mean(image, dim=(1, 2))
    # Add standard deviation of each channel to the std tensor
    std += torch.std(image, dim=(1, 2))

# Calculate mean and standard deviation for the entire dataset
mean /= len(train_dataset)
std /= len(train_dataset)

print("Mean values for each channel:", mean)
print("Standard deviation values for each channel:", std)

# Define data transformations with normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),                # Resize images to 224x224 pixels
    transforms.ToTensor(),                         # Convert images to PyTorch tensors
    transforms.Normalize(mean, std)                # Normalize images using computed mean and std
])

# Update the datasets with the new transformations
train_dataset = ImageFolder(root='./CUB_dataset/train', transform=transform)
test_dataset = ImageFolder(root='./CUB_dataset/test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the pre-trained ResNet18 model
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Adapt the fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')


Mean values for each channel: tensor([0.4875, 0.5013, 0.4342])
Standard deviation values for each channel: tensor([0.1749, 0.1738, 0.1861])
Epoch [1/10], Loss: 4.851184210343795
Epoch [2/10], Loss: 3.3866179206154565
Epoch [3/10], Loss: 2.448948258486661
Epoch [4/10], Loss: 1.8382717362317171
Epoch [5/10], Loss: 1.4159797137433832
Epoch [6/10], Loss: 1.1049679084257646
Epoch [7/10], Loss: 0.8677346444129944
Epoch [8/10], Loss: 0.67312242009423
Epoch [9/10], Loss: 0.5192882733995264
Epoch [10/10], Loss: 0.4058974406935952
Accuracy on test set: 69.80%
